## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from sklearn import metrics

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']].copy()
y_df.head()


,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [4]:
# Create a list of at least 10 column names to use as X data, I picked 12 columns
X_cols = ['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction', 
          'OverTime', 'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany', 
          'YearsSinceLastPromotion', 'NumCompaniesWorked', 'EnvironmentSatisfaction',
          'JobInvolvement']

# Create X_df using your selected columns
X_df = attrition_df[X_cols].copy()

# Show the data types for X_df
X_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Education                1470 non-null   int64 
 1   Age                      1470 non-null   int64 
 2   DistanceFromHome         1470 non-null   int64 
 3   JobSatisfaction          1470 non-null   int64 
 4   OverTime                 1470 non-null   object
 5   StockOptionLevel         1470 non-null   int64 
 6   WorkLifeBalance          1470 non-null   int64 
 7   YearsAtCompany           1470 non-null   int64 
 8   YearsSinceLastPromotion  1470 non-null   int64 
 9   NumCompaniesWorked       1470 non-null   int64 
 10  EnvironmentSatisfaction  1470 non-null   int64 
 11  JobInvolvement           1470 non-null   int64 
dtypes: int64(11), object(1)
memory usage: 137.9+ KB


In [5]:
y_df['Attrition'].value_counts()

Attrition
No     1233
Yes     237
Name: count, dtype: int64

In [6]:
y_df['Department'].value_counts()

Department
Research & Development    961
Sales                     446
Human Resources            63
Name: count, dtype: int64

In [7]:
X_df['OverTime'].value_counts()

OverTime
No     1054
Yes     416
Name: count, dtype: int64

In [8]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_df['OverTime'] = X_df['OverTime'].astype("category").cat.codes
X_df['OverTime'].value_counts()

OverTime
0    1054
1     416
Name: count, dtype: int64

In [9]:
# Split the data into training and testing sets - I moved this line of code below converting X data to numeric
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=1)


In [10]:
# Create a StandardScaler and fit to the training data
scaler = StandardScaler().fit(X_train)

# Scale the training and testing data
# Fit the scaler to the features training dataset
X_train_scaled = scaler.transform(X_train)

# Fit the scaler to the features testing dataset
X_test_scaled = scaler.transform(X_test)

In [11]:
# Create a OneHotEncoder for the Department column
department_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
department_train = department_encoder.fit_transform(y_train[['Department']])
department_columns = department_encoder.get_feature_names_out(['Department'])
y_train_department = pd.DataFrame(department_train, columns=department_columns)
y_train_department

# Create two new variables by applying the encoder
# to the training and testing data
department_test = department_encoder.fit_transform(y_test[['Department']])
department_columns = department_encoder.get_feature_names_out(['Department'])
y_test_department = pd.DataFrame(department_test, columns=department_columns)
y_test_department


,Department_Human Resources,Department_Research & Development,Department_Sales
0,0.0,1.0,0.0
1,0.0,0.0,1.0
2,0.0,1.0,0.0
3,0.0,0.0,1.0
4,0.0,1.0,0.0
...,...,...,...
363,0.0,1.0,0.0
364,0.0,1.0,0.0
365,0.0,1.0,0.0
366,0.0,1.0,0.0


In [12]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
attrition_train = attrition_encoder.fit_transform(y_train[['Attrition']])
attrition_columns = attrition_encoder.get_feature_names_out(['Attrition'])
y_train_attrition = pd.DataFrame(attrition_train, columns=attrition_columns)
y_train_attrition

# Create two new variables by applying the encoder
# to the training and testing data
attrition_test = attrition_encoder.fit_transform(y_test[['Attrition']])
attrition_columns = attrition_encoder.get_feature_names_out(['Attrition'])
y_test_attrition = pd.DataFrame(attrition_test, columns=attrition_columns)
y_test_attrition

,Attrition_No,Attrition_Yes
0,0.0,1.0
1,0.0,1.0
2,0.0,1.0
3,1.0,0.0
4,1.0,0.0
...,...,...
363,1.0,0.0
364,1.0,0.0
365,1.0,0.0
366,0.0,1.0


## Create, Compile, and Train the Model

In [13]:
# Find the number of columns in the X training data
X_train.shape

# Create the input layer
input_layer = layers.Input(shape=(X_train.shape[1],), name='input_features')

# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu')(input_layer)
shared_layer2 = layers.Dense(128, activation='relu')(shared_layer1)

In [14]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden_layer = layers.Dense(32, activation='relu')(shared_layer2)

# Create the output layer
department_output = layers.Dense(len(y_train_department.columns), activation='softmax', name='department_output')(department_hidden_layer)


In [15]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden_layer = layers.Dense(32, activation='relu')(shared_layer2)

# Create the output layer
attrition_output = layers.Dense(len(y_train_attrition.columns), activation='sigmoid', name='attrition_output')(attrition_hidden_layer)


In [16]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy', 'attrition_output': 'binary_crossentropy'},
              metrics={'department_output': 'accuracy', 'attrition_output': 'accuracy'})


# Summarize the model
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_features (InputLayer)   │ (None, 12)                │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, 64)                │             832 │ input_features[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_1 (Dense)               │ (None, 128)               │           8,320 │ dense[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_2 (Dense)               │ (None, 32)                │           4,128 │ dense_1[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_3 (Dense)               │ (None, 32)                │           4,128 │ dense_1[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ department_output (Dense)     │ (None, 3)                 │              99 │ dense_2[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ attrition_output (Dense)      │ (None, 2)                 │              66 │ dense_3[0][0]              │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 17,573 (68.64 KB)

 Trainable params: 17,573 (68.64 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
# Train the model
model.fit(
    X_train,
    {'department_output': y_train_department, 'attrition_output': y_train_attrition},
    epochs=100,
    batch_size=32,
    validation_split=0.2
)


Epoch 1/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - attrition_output_accuracy: 0.5269 - department_output_accuracy: 0.6288 - loss: 1.7731 - val_attrition_output_accuracy: 0.8281 - val_department_output_accuracy: 0.6425 - val_loss: 1.3182
Epoch 2/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - attrition_output_accuracy: 0.8558 - department_output_accuracy: 0.6644 - loss: 1.2420 - val_attrition_output_accuracy: 0.8281 - val_department_output_accuracy: 0.6380 - val_loss: 1.3909
Epoch 3/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.8571 - department_output_accuracy: 0.6523 - loss: 1.2032 - val_attrition_output_accuracy: 0.8281 - val_department_output_accuracy: 0.6335 - val_loss: 1.3032
Epoch 4/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - attrition_output_accuracy: 0.8449 - department_output_accuracy: 0.6687 - loss: 1.1876 - val_attrition_output_accuracy: 0.8281 - val_department_output_accuracy: 0.6018 - val_loss: 1.2976
Epoch 5/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 4

In [18]:
# Evaluate the model with the testing data
test_results = model.evaluate(X_test, {'department_output': y_test_department, 'attrition_output': y_test_attrition})
test_results

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8036 - department_output_accuracy: 0.5399 - loss: 1.6872 


[1.5884296894073486, 0.8288043737411499, 0.570652186870575]

In [19]:
# Print the accuracy for both department and attrition
print('Attrition Predictions Accuracy: %.3f' % test_results[1])
print('Department Predictions Accuracy: %.3f' % test_results[2])

Attrition Predictions Accuracy: 0.829
Department Predictions Accuracy: 0.571


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. I think accuracy is a good metric to start with for this data, because we are predicting classification for Department and Attrition. But because the input data is skewed (see #3 below), other metrics like F1-Score, Precision, Recall, MultiClassConfusionMatrixPlot, etc. may also be useful.
1. 
I chose "softmax" activation function for Department output layer, because there were 3 values for Department (a) - Research & Development(b) , Sal,es an(c) d Human Resources. I chose "sigmoid" activation function for Attrition output layer, because there were only 2 possible values for Attrition a binary classification of - Y/d N
.1. 

Obtain additional data that would help predict Department better, for example, college major, professional certifications, etc. Also input data was very skewed with respect to Department, only 63 rows or 4% of data was for Department Human Resources, 446 ror 30% ows for Sal30% and remaining 66% of rows were Department R&D. The model might predict better with more data rows for HR and Sales Departme
3. 